In [19]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from environments import RodentSingleClipTrack
import numpy as np
import mediapy as media
import jax.numpy as jp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
params = {
    "scale_factor": .9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 3,
    "clip_path": "12_22_1_250_clip_0.p",
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
    ],
}

# Creating env objects

In [21]:
env = RodentSingleClipTrack(params)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitatio

In [22]:
key = random.PRNGKey(0)
jit_step = jax.jit(env.step)
jit_reset = jax.jit(env.reset)
# next = env.step(state, random.uniform(key, shape=(env.sys.nu,)))

In [24]:
type(env._ref_traj)

dict

In [7]:
n = env.reset(key)

KeyError: 'position'

In [ ]:
next = jit_reset(key)

rollout = []
for _ in range(100):
    next = jit_step(next, random.uniform(key, shape=(env.sys.nu,)))


In [ ]:
res = jax.tree_util.tree_map(
    lambda x: jax.lax.dynamic_slice_in_dim(x, 3, 4, axis=1),
    env._ref_traj,
)

In [ ]:
jax.tree_util.tree_map(
    lambda x: print(x.shape),
    env._ref_traj,
)

In [ ]:
env._ref_traj.body_positions[:,env.body_idxs]

In [ ]:
env._ref_traj.angular_velocity.shape

In [ ]:
# def slice_arrays(start_index, end_index):
#     def slicer(x):
#         if isinstance(x, jp.ndarray):
#             return x[..., start_index:end_index]
#         return x
#     return slicer

# sliced_tree = jax.tree_map(slice_arrays(0, 5), env._ref_traj)
# sliced_tree.body_positions.shape

In [ ]:
start_index = 0
end_index = 5
sliced_clip = env._ref_traj.slice_clip(start_index, end_index)
sliced_clip.center_of_mass.shape

# Creating state objects & reset_fn, get_obs_fn
`reset_fn` and `get_obs_fn` is called when initilizing the environment and creating the state object

In [ ]:
rng = random.PRNGKey(0)
state = env.reset(rng)

In [ ]:
state.info['start_frame']

In [ ]:
env._ref_traj.velocity[:, state.info['start_frame']]

In [ ]:
state.pipeline_state.xpos.shape

In [ ]:
env._ref_traj.body_positions.shape

In [ ]:
state.obs.shape

end effector position

In [ ]:
state.pipeline_state.xpos[jp.array(env._end_eff_idx)]

In [ ]:
env._ref_traj.end_effectors.shape

# Creating step_fn, reward_fn, & termination_fn
`step_fn`, which calls `reward_fn` & `termination_fn` is only used when actually making action, not when setting up environment and state object

In [ ]:
# rollout = [state.pipeline_state]
# ctrl = jax.numpy.array(np.random.uniform(-1,1, env.sys.nu))
# state = env.step(state, ctrl)
# rollout.append(state.pipeline_state)

In [ ]:
# media.show_video(env.render(rollout,camera='close_profile'), fps=1.0 / env.dt)